In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [ ]:
data_tukang = pd.read_csv('data_tukang.csv')
data_user = pd.read_csv('data_user.csv')

In [ ]:
tf_user = data_user.loc[:, ['Nama','Daerah','Order', 'Rating', 'Keterangan', 'Timestamp']]
tf_user = tf_user.rename(columns={'Nama': 'Nama User', 'Rating': 'User Rating'})
tf_user

,Nama User,Daerah,Order,User Rating,Keterangan,Timestamp
0,Andi Susanto,Purwokerto,Ningsih Instalasi Pipa,2.8,Pemasangan Pipa Sumur,20/12/2023
1,Andi Susanto,Purwokerto,Kurniawan Tukang Kayu,4.2,Servis Dipan Kasur,8/7/2023
2,Andi Susanto,Purwokerto,Lestari Tukang Bangunan dan Pengecetan,4.9,Cat Tembok,5/3/2023
3,Andi Susanto,Purwokerto,Lestari Instalasi Listrik,4.1,Service Kulkas,10/12/2023
4,Andi Susanto,Purwokerto,Sari Design dan Cetak,2.8,Design Pamflet dan Cetak,11/8/2023
...,...,...,...,...,...,...
155,Siti Wati,Semarang,Utami Tukang Aluminium,4.7,Pemasangan Plat Mesin,7/3/2023
156,Siti Wati,Semarang,Kusuma Tukang Kayu,2.8,Pembuatakn Lemari,21/8/2023
157,Siti Wati,Semarang,Susanto Perpipaan Gas,4.5,Perawatan Pipa,20/9/2023
158,Siti Wati,Semarang,Hartono Instalasi Pipa,4.9,Perbaikan Pipa,10/9/2023


In [ ]:
tf_tukang = data_tukang.loc[:, ['Nama','Daerah','Tempat Pekerjaan','Keahlian', 'Harga', 'Rating', 'Pengrating']]
tf_tukang = tf_tukang.rename(columns={'Nama': 'Nama Tukang', 'Tempat Pekerjaan': 'Order','Rating': 'Rating Tukang', 'Pengrating': 'Jumlah Rating'})
tf_tukang

,Nama Tukang,Daerah,Order,Keahlian,Harga,Rating Tukang,Jumlah Rating
0,Ahmad Lestari,Sleman,Lestari Tukang Bangunan dan Pengecetan,Tukang Bangunan dan Pengecetan,95000,4.7,87
1,Wati Wibowo,Kebumen,Wibowo Tukang Aluminium,Tukang Aluminium,60000,4.7,80
2,Adi Sari,Semarang,Sari Tukang AC,Tukang AC,150000,4.8,20
3,Joko Saputra,Banyuwangi,Saputra Instalasi Pipa,Instalasi Pipa,70000,3.5,33
4,Sinta Gunawan,Wonosobo,Gunawan Tukang Listrik Rumah Tangga,Tukang Listrik Rumah Tangga,120000,3.5,1
...,...,...,...,...,...,...,...
995,Andi Susanto,Semarang,Susanto Design Logo,Design Logo,95000,4.8,14
996,Dewa Wijaya,Tasikmalaya,Wijaya Tukang Bangunan dan Pengecetan,Tukang Bangunan dan Pengecetan,70000,4.8,14
997,Rahayu Gunawan,Solo,Gunawan Tukang AC,Tukang AC,80000,4.3,14
998,Adi Hartono,Purwokerto,Hartono Tukang Listrik Rumah Tangga,Tukang Listrik Rumah Tangga,130000,4.8,46


In [ ]:
def cari_tukang(nama_user, kebutuhan, tf_user, tf_tukang):
    # Mengambil daerah dari data berdasarkan nama pengguna
    lokasi_user = tf_user.loc[tf_user['Nama User'] == nama_user, 'Daerah'].values[0]

    # Menggunakan pemrosesan teks sederhana untuk pencocokan kebutuhan pengguna
    matching_jobs = tf_tukang[tf_tukang['Keahlian'].str.contains(kebutuhan, case=False)]

    # Memfilter berdasarkan lokasi (daerah dari data pengguna)
    matching_jobs = matching_jobs[matching_jobs['Daerah'] == lokasi_user]

    # Fitur yang akan digunakan untuk peringkat
    features = matching_jobs[['Rating Tukang', 'Jumlah Rating','Harga']]

    # Inisialisasi model KNN
    knn = NearestNeighbors(n_neighbors=len(matching_jobs))  # Jumlah tetangga = jumlah tukang yang cocok

    # Latih model dengan fitur yang dipilih
    knn.fit(features)

    # Mendapatkan indeks tukang yang cocok
    _, indices = knn.kneighbors(features)

    # Menampilkan hasil berdasarkan peringkat dari model
    hasil_terurut = matching_jobs.iloc[indices[0]]  # Menggunakan indeks dari model KNN

    # Mengurutkan hasil berdasarkan Rating Tukang secara descending
    hasil_terurut = hasil_terurut.sort_values(by='Rating Tukang', ascending=False)

    return hasil_terurut

In [ ]:
# Input dari pengguna
nama_user = input("Masukkan nama Anda: ")
kebutuhan_user = input("Masukkan jenis pekerjaan yang dibutuhkan: ")

# Memanggil fungsi untuk mencari tukang
hasil_pencarian = cari_tukang(nama_user, kebutuhan_user, tf_user, tf_tukang)

# Menampilkan hasil tukang yang paling sesuai dengan urutan rating descending
print("Berikut adalah tukang yang paling sesuai dengan kebutuhan Anda di", hasil_pencarian['Daerah'].iloc[0], ":")
hasil_pencarian[['Nama Tukang', 'Daerah', 'Order', 'Keahlian', 'Harga', 'Rating Tukang', 'Jumlah Rating']]

Masukkan nama Anda: Hendra Setiawan
Masukkan jenis pekerjaan yang dibutuhkan: gas
Berikut adalah tukang yang paling sesuai dengan kebutuhan Anda di Wonosobo :


,Nama Tukang,Daerah,Order,Keahlian,Harga,Rating Tukang,Jumlah Rating
15,Rina Susanto,Wonosobo,Susanto Perpipaan Gas,Perpipaan Gas,70000,4.7,95
325,Lina Pratama,Wonosobo,Pratama Perpipaan Gas,Perpipaan Gas,120000,4.7,2
281,Fina Saputri,Wonosobo,Saputri Perpipaan Gas,Perpipaan Gas,150000,4.7,5
602,Sari Gunawan,Wonosobo,Gunawan Perpipaan Gas,Perpipaan Gas,120000,4.6,99
908,Rudi Hartono,Wonosobo,Hartono Perpipaan Gas,Perpipaan Gas,140000,4.6,20
420,Reza Kusuma,Wonosobo,Kusuma Perpipaan Gas,Perpipaan Gas,130000,3.5,17
633,Rahayu Susanto,Wonosobo,Susanto Perpipaan Gas,Perpipaan Gas,60000,3.2,17
188,Rudi Wibowo,Wonosobo,Wibowo Perpipaan Gas,Perpipaan Gas,80000,2.8,17


In [ ]:
import pandas as pd
import pandas as pd
import nltk
from datetime import datetime
from itertools import combinations
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Data keywords
keywords = [
    'Tukang Bangunan dan Pengecetan', 'Instalasi Listrik', 'Tukang Kayu', 'Tukang AC', 'Instalasi Pipa',
    'Tukang Aluminium', 'Instalasi Pipa', 'Perpipaan Gas', 'Listrik Rumah Tangga',
    'Design Logo', 'Design dan Cetak', 'Design dan Fotografi', 'Foto dan Videografi'
]

# Fungsi untuk mengelompokkan order berdasarkan kata kunci
def group_orders_by_keyword(order):
    for keyword in keywords:
        if keyword in order:
            return keyword
    # return 'Other'  # Jika tidak cocok dengan keyword apapun, masukkan ke grup "Other"

# Menambahkan kolom 'Keyword' ke DataFrame
tf_user['Keyword'] = tf_user['Order'].apply(group_orders_by_keyword)

# Menambahkan kolom 'Row Number' yang berisi nomor baris sebelumnya
tf_user['Row Number'] = tf_user.index + 1  # Nomor baris dimulai dari 1, bukan 0

# Melakukan pengelompokkan berdasarkan user dan keyword
grouped_data = tf_user.groupby(['Nama User', 'Keyword', 'Order'])['Row Number'].apply(list).reset_index(name='Row Numbers')

# Filter data hanya untuk yang memiliki lebih dari 1 pesanan
filtered_data = grouped_data[grouped_data['Row Numbers'].apply(len) > 1]
filtered_data


,Nama User,Keyword,Order,Row Numbers
3,Andi Susanto,Instalasi Listrik,Lestari Instalasi Listrik,"[4, 6]"
5,Andi Susanto,Perpipaan Gas,Kurniawan Perpipaan Gas,"[10, 11]"
7,Andi Susanto,Tukang Kayu,Kurniawan Tukang Kayu,"[2, 8]"
21,Budi Saputra,Design dan Cetak,Susanto Design dan Cetak,"[24, 25, 27]"
26,Budi Saputra,Tukang Bangunan dan Pengecetan,Kurniawan Tukang Bangunan dan Pengecetan,"[26, 28]"
32,Dewi Sari,Tukang Aluminium,Hartono Tukang Aluminium,"[34, 35, 36]"
36,Dian Ningsih,Design Logo,Setiawan Design Logo,"[48, 50]"
37,Dian Ningsih,Design dan Cetak,Wibowo Design dan Cetak,"[46, 47]"
38,Dian Ningsih,Foto dan Videografi,Purba Foto dan Videografi,"[49, 51, 53, 54]"
49,Eko Lestari,Tukang Bangunan dan Pengecetan,Pratama Tukang Bangunan dan Pengecetan,"[64, 65]"


In [ ]:
# Ambil kolom 'Row Numbers' dari filtered_data
row_numbers = filtered_data['Row Numbers']

# Membuat dictionary kosong untuk menyimpan set kata yang ditemukan berdasarkan indeks
kata_per_indeks = {}

# Melakukan perulangan untuk setiap indeks pada kolom 'Row Numbers'
for i, indices in enumerate(row_numbers, start=1):
    kata_yang_sama = set()  # Inisialisasi set kata untuk setiap iterasi
    rows_with_same_words = set()  # Inisialisasi set untuk menyimpan nomor baris yang memiliki kata yang sama

    for index in indices:
        # Mendapatkan nilai 'keterangan' pada tf_user berdasarkan indeks dari 'Row Numbers'
        keterangan = tf_user.loc[index-1, 'Keterangan']

        # Tokenisasi kata dalam 'keterangan'
        kata = word_tokenize(keterangan.lower())  # Ubah ke huruf kecil dan tokenisasi kata

        # Tambahkan kata ke set kata_yang_sama
        kata_yang_sama.update(kata)
        rows_with_same_words.add(index)  # Tambahkan nomor baris yang memiliki kata yang sama

    # Simpan set kata yang ditemukan ke dalam dictionary berdasarkan indeksnya beserta data barisnya
    kata_per_indeks[f"Set_{i}"] = {'Kata yang Sama': list(kata_yang_sama), 'Rows': list(rows_with_same_words)}

# Mengumpulkan informasi ke dalam DataFrame
data = []
for key, value in kata_per_indeks.items():
    for index, row in tf_user.iterrows():
        if index+1 in value['Rows']:
          data.append([row['Nama User'], value['Kata yang Sama'], value['Rows']])

# Membuat DataFrame dari data yang terkumpul
df = pd.DataFrame(data, columns=['Nama User', 'Kata Sama', 'Row Numbers'])
# Convert lists in 'Kata Sama' and 'Row Numbers' columns to tuples
df['Kata Sama'] = df['Kata Sama'].apply(tuple)
df['Row Numbers'] = df['Row Numbers'].apply(tuple)

# Drop duplicates based on columns 'Nama User', 'Kata Sama', and 'Row Numbers'
df_filtered = df.drop_duplicates(subset=['Nama User', 'Kata Sama', 'Row Numbers'])

# Display the DataFrame without duplicates
df_filtered

,Nama User,Kata Sama,Row Numbers
0,Andi Susanto,"(service, kulkas)","(4, 6)"
2,Andi Susanto,"(pipa, perwatan)","(10, 11)"
4,Andi Susanto,"(dipan, kursi, servis, meja, kasur, dan)","(8, 2)"
6,Budi Saputra,"(desaing, kalender, desain, cetak, banner, dan)","(24, 25, 27)"
9,Budi Saputra,"(tembok, cat)","(26, 28)"
11,Dewi Sari,"(pembuatan, plat, perawatan, pemasangan)","(34, 35, 36)"
14,Dian Ningsih,"(desain, logo, event)","(48, 50)"
16,Dian Ningsih,"(cetak, banner)","(46, 47)"
18,Dian Ningsih,"(wisuda, keluarga, foto, staff)","(49, 51, 53, 54)"
22,Eko Lestari,"(tembok, cat)","(64, 65)"


In [ ]:
!pip install Sastrawi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
# Ambil kolom 'Row Numbers' dari filtered_data
row_numbers = filtered_data['Row Numbers']

# Membuat dictionary kosong untuk menyimpan set kata yang ditemukan berdasarkan indeks
kata_per_indeks = {}

# Inisialisasi Sastrawi untuk stop words removal dan stemming
stopword_factory = StopWordRemoverFactory()
stemmer_factory = StemmerFactory()

stopwords = stopword_factory.get_stop_words()
stemmer = stemmer_factory.create_stemmer()

# Melakukan perulangan untuk setiap indeks pada kolom 'Row Numbers'
for i, indices in enumerate(row_numbers, start=1):
    kata_yang_sama = set()  # Inisialisasi set kata untuk setiap iterasi
    rows_with_same_words = set()  # Inisialisasi set untuk menyimpan nomor baris yang memiliki kata yang sama

    for index in indices:
        # Mendapatkan nilai 'keterangan' pada tf_user berdasarkan indeks dari 'Row Numbers'
        keterangan = tf_user.loc[index-1, 'Keterangan']

        # Tokenisasi kata dalam 'keterangan' dengan N-Gram (misalnya, bigram - N=2)
        tokens = word_tokenize(keterangan.lower())
        cleaned_tokens = [stemmer.stem(token) for token in tokens if token.isalpha() and token not in stopwords]
        ngram_tokens = [' '.join(cleaned_tokens[i:i+2]) for i in range(len(cleaned_tokens)-1)]  # Ubah range sesuai dengan nilai N

        # Tambahkan token N-Gram ke set kata_yang_sama
        kata_yang_sama.update(ngram_tokens)
        rows_with_same_words.add(index)  # Tambahkan nomor baris yang memiliki kata yang sama

    # Simpan set kata yang ditemukan ke dalam dictionary berdasarkan indeksnya beserta data barisnya
    kata_per_indeks[f"Set_{i}"] = {'Kata yang Sama': list(kata_yang_sama), 'Rows': list(rows_with_same_words)}

# Mengumpulkan informasi ke dalam DataFrame
data = []
for key, value in kata_per_indeks.items():
    for index, row in tf_user.iterrows():
        if index+1 in value['Rows']:
            data.append([row['Nama User'], value['Kata yang Sama'], value['Rows']])

# Membuat DataFrame dari data yang terkumpul
df = pd.DataFrame(data, columns=['Nama User', 'Kata Sama', 'Row Numbers'])
# Convert lists in 'Kata Sama' and 'Row Numbers' columns to tuples
df['Kata Sama'] = df['Kata Sama'].apply(tuple)
df['Row Numbers'] = df['Row Numbers'].apply(tuple)

# Drop duplicates based on columns 'Nama User', 'Kata Sama', and 'Row Numbers'
df_filtered = df.drop_duplicates(subset=['Nama User', 'Kata Sama', 'Row Numbers'])

# Display the DataFrame without duplicates
df_filtered


,Nama User,Kata Sama,Row Numbers
0,Andi Susanto,"(service kulkas,)","(4, 6)"
2,Andi Susanto,"(watan pipa,)","(10, 11)"
4,Andi Susanto,"(meja kursi, servis meja, servis dipan, dipan ...","(8, 2)"
6,Budi Saputra,"(cetak banner, desaing cetak, desain cetak, ce...","(24, 25, 27)"
9,Budi Saputra,"(cat tembok,)","(26, 28)"
11,Dewi Sari,"(buat plat, pasang plat, awat plat)","(34, 35, 36)"
14,Dian Ningsih,"(desain logo, logo event)","(48, 50)"
16,Dian Ningsih,"(cetak banner,)","(46, 47)"
18,Dian Ningsih,"(foto staff, foto wisuda, foto keluarga)","(49, 51, 53, 54)"
22,Eko Lestari,"(cat tembok,)","(64, 65)"


In [ ]:
# Membuat dictionary kosong untuk menyimpan nilai mutlak dari selisih waktu
absolute_time_diffs_dict = {}

# Melakukan perulangan untuk setiap kelompok 'Row Numbers'
for i, indices in enumerate(df_filtered['Row Numbers'], start=1):
    # Menghasilkan kombinasi unik dari indeks dalam setiap kelompok 'Row Numbers'
    index_combinations = combinations(indices, 2)  # Mengambil kombinasi 2 indeks untuk mendapatkan selisih waktu

    # Inisialisasi list untuk menyimpan nilai mutlak dari selisih waktu dalam detik
    absolute_time_diffs = []

    # Iterasi melalui setiap kombinasi indeks
    for index_pair in index_combinations:
        # Mendapatkan timestamp dari tf_user berdasarkan indeks
        timestamp1 = datetime.strptime(tf_user.loc[index_pair[0] - 1, 'Timestamp'], '%d/%m/%Y')
        timestamp2 = datetime.strptime(tf_user.loc[index_pair[1] - 1, 'Timestamp'], '%d/%m/%Y')

        # Menghitung nilai mutlak dari selisih waktu dalam detik
        abs_diff_seconds = abs((timestamp2 - timestamp1).total_seconds())
        absolute_time_diffs.append(abs_diff_seconds)  # Menyimpan nilai mutlak dari selisih waktu dalam detik

    # Simpan nilai mutlak dari selisih waktu ke dalam dictionary berdasarkan indeks kelompoknya
    absolute_time_diffs_dict[f"Set_{i}"] = absolute_time_diffs

# Menampilkan nilai mutlak dari selisih waktu antar indeks untuk setiap kelompok
for key, value in absolute_time_diffs_dict.items():
    print(f"Kelompok {key}: Nilai Mutlak Selisih Waktu: {value}")


Kelompok Set_1: Nilai Mutlak Selisih Waktu: [13219200.0]
Kelompok Set_2: Nilai Mutlak Selisih Waktu: [4838400.0]
Kelompok Set_3: Nilai Mutlak Selisih Waktu: [13651200.0]
Kelompok Set_4: Nilai Mutlak Selisih Waktu: [8553600.0, 9936000.0, 1382400.0]
Kelompok Set_5: Nilai Mutlak Selisih Waktu: [14947200.0]
Kelompok Set_6: Nilai Mutlak Selisih Waktu: [21081600.0, 15638400.0, 5443200.0]
Kelompok Set_7: Nilai Mutlak Selisih Waktu: [2160000.0]
Kelompok Set_8: Nilai Mutlak Selisih Waktu: [18403200.0]
Kelompok Set_9: Nilai Mutlak Selisih Waktu: [8380800.0, 3801600.0, 3628800.0, 4579200.0, 4752000.0, 172800.0]
Kelompok Set_10: Nilai Mutlak Selisih Waktu: [9936000.0]
Kelompok Set_11: Nilai Mutlak Selisih Waktu: [5443200.0]
Kelompok Set_12: Nilai Mutlak Selisih Waktu: [23587200.0, 14947200.0, 31363200.0, 8640000.0, 7776000.0, 16416000.0]
Kelompok Set_13: Nilai Mutlak Selisih Waktu: [12182400.0]
Kelompok Set_14: Nilai Mutlak Selisih Waktu: [29721600.0]
Kelompok Set_15: Nilai Mutlak Selisih Waktu: [

In [ ]:
# Membuat dictionary kosong untuk menyimpan tanggal maksimal dari setiap kelompok indeks
tanggal_maksimal_per_kelompok = {}

# Melakukan perulangan untuk setiap kelompok 'Row Numbers'
for i, indices in enumerate(df_filtered['Row Numbers'], start=1):
    # Inisialisasi variabel untuk menyimpan tanggal maksimal dalam satu kelompok
    tanggal_maksimal = None

    # Iterasi melalui setiap indeks dalam kelompok
    for index in indices:
        # Mendapatkan timestamp dari tf_user berdasarkan indeks
        timestamp = datetime.strptime(tf_user.loc[index - 1, 'Timestamp'], '%d/%m/%Y')

        # Membandingkan dan mengupdate tanggal maksimal
        if tanggal_maksimal is None or timestamp > tanggal_maksimal:
            tanggal_maksimal = timestamp

    # Simpan tanggal maksimal ke dalam dictionary berdasarkan kelompoknya
    tanggal_maksimal_per_kelompok[f"Set_{i}"] = tanggal_maksimal

# Membuat DataFrame dari tanggal_maksimal_per_kelompok
df_tanggal_maksimal = pd.DataFrame({
    'Nama User': df_filtered['Nama User'],
    'Tanggal Maksimal': [value.strftime('%d/%m/%Y') if value else 'Data tidak tersedia' for value in tanggal_maksimal_per_kelompok.values()]
})

# Menampilkan DataFrame
df_tanggal_maksimal


,Nama User,Tanggal Maksimal
0,Andi Susanto,10/12/2023
2,Andi Susanto,30/08/2023
4,Andi Susanto,13/12/2023
6,Budi Saputra,03/11/2023
9,Budi Saputra,31/12/2023
11,Dewi Sari,12/12/2023
14,Dian Ningsih,10/10/2023
16,Dian Ningsih,20/09/2023
18,Dian Ningsih,10/07/2023
22,Eko Lestari,03/11/2023


In [ ]:
# Membuat dictionary kosong untuk menyimpan nilai mutlak dari selisih waktu berdasarkan kata yang sama
absolute_time_diffs_per_word = defaultdict(list)

# Iterasi untuk setiap baris dalam df_filtered
for index, row in df_filtered.iterrows():
    # Mendapatkan kata yang sama dari baris saat ini sebagai satu kesatuan
    kata_sama = row['Kata Sama']

    # Menghasilkan kombinasi unik dari indeks dalam setiap kelompok 'Row Numbers'
    index_combinations = combinations(row['Row Numbers'], 2)  # Mengambil kombinasi 2 indeks untuk mendapatkan selisih waktu

    # Inisialisasi list untuk menyimpan nilai mutlak dari selisih waktu dalam detik
    absolute_time_diffs = []

    # Iterasi melalui setiap kombinasi indeks
    for index_pair in index_combinations:
        # Mendapatkan timestamp dari tf_user berdasarkan indeks
        timestamp1 = datetime.strptime(tf_user.loc[index_pair[0] - 1, 'Timestamp'], '%d/%m/%Y')
        timestamp2 = datetime.strptime(tf_user.loc[index_pair[1] - 1, 'Timestamp'], '%d/%m/%Y')

        # Menghitung nilai mutlak dari selisih waktu dalam detik
        abs_diff_seconds = abs((timestamp2 - timestamp1).total_seconds())
        absolute_time_diffs.append(abs_diff_seconds)  # Menyimpan nilai mutlak dari selisih waktu dalam detik

    # Menambahkan nilai mutlak dari selisih waktu ke dalam dictionary berdasarkan kata yang sama
    for word in kata_sama:
        absolute_time_diffs_per_word[word].extend(absolute_time_diffs)

# Menampilkan nilai mutlak dari selisih waktu antar indeks untuk setiap kata yang sama
for word, time_diffs in absolute_time_diffs_per_word.items():
    print(f"Kata Sama: {word}, Nilai Mutlak Selisih Waktu: {time_diffs}")


Kata Sama: service kulkas, Nilai Mutlak Selisih Waktu: [13219200.0]
Kata Sama: watan pipa, Nilai Mutlak Selisih Waktu: [4838400.0]
Kata Sama: meja kursi, Nilai Mutlak Selisih Waktu: [13651200.0]
Kata Sama: servis meja, Nilai Mutlak Selisih Waktu: [13651200.0]
Kata Sama: servis dipan, Nilai Mutlak Selisih Waktu: [13651200.0]
Kata Sama: dipan kasur, Nilai Mutlak Selisih Waktu: [13651200.0]
Kata Sama: cetak banner, Nilai Mutlak Selisih Waktu: [8553600.0, 9936000.0, 1382400.0, 18403200.0]
Kata Sama: desaing cetak, Nilai Mutlak Selisih Waktu: [8553600.0, 9936000.0, 1382400.0]
Kata Sama: desain cetak, Nilai Mutlak Selisih Waktu: [8553600.0, 9936000.0, 1382400.0]
Kata Sama: cetak kalender, Nilai Mutlak Selisih Waktu: [8553600.0, 9936000.0, 1382400.0]
Kata Sama: cat tembok, Nilai Mutlak Selisih Waktu: [14947200.0, 9936000.0, 23587200.0, 14947200.0, 31363200.0, 8640000.0, 7776000.0, 16416000.0, 29721600.0]
Kata Sama: buat plat, Nilai Mutlak Selisih Waktu: [21081600.0, 15638400.0, 5443200.0]
Kat

In [ ]:
# Buat dictionary kosong untuk menyimpan rata-rata selisih waktu per kata
average_time_diffs_per_word = {}

# Hitung rata-rata dari nilai mutlak selisih waktu untuk setiap kata yang sama
for word, time_diffs in absolute_time_diffs_per_word.items():
    average_time_diffs_per_word[word] = sum(time_diffs) / len(time_diffs)

# Buat DataFrame dari dictionary average_time_diffs_per_word
df_average_time_diffs = pd.DataFrame(list(average_time_diffs_per_word.items()), columns=['Kata Sama', 'Selisih Waktu'])


df_average_time_diffs['Cluster']=df_average_time_diffs.index

df_average_time_diffs

,Kata Sama,Selisih Waktu,Cluster
0,service kulkas,1.321920e+07,0
1,watan pipa,4.838400e+06,1
2,meja kursi,1.365120e+07,2
3,servis meja,1.365120e+07,3
4,servis dipan,1.365120e+07,4
5,dipan kasur,1.365120e+07,5
6,cetak banner,9.568800e+06,6
7,desaing cetak,6.624000e+06,7
8,desain cetak,6.624000e+06,8
9,cetak kalender,6.624000e+06,9


In [ ]:
# Download stopwords jika belum
nltk.download('stopwords')

# Fungsi untuk membersihkan teks
def preprocess_text(text):
    stop_words = set(stopwords.words('indonesian'))
    words = nltk.word_tokenize(text.lower())
    return ' '.join([word for word in words if word.isalpha() and word not in stop_words])

# Membersihkan teks di kolom 'keterangan' dan 'kata sama'
tf_user['Cleaned_Keterangan'] = tf_user['Keterangan'].apply(preprocess_text)
df_average_time_diffs['Cleaned_KataSama'] = df_average_time_diffs['Kata Sama'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Menginisialisasi objek TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Menghitung vektor TF-IDF untuk kolom 'Cleaned_Keterangan' dan 'Cleaned_KataSama'
tfidf_keterangan = tfidf_vectorizer.fit_transform(tf_user['Cleaned_Keterangan'])
tfidf_kata_sama = tfidf_vectorizer.transform(df_average_time_diffs['Cleaned_KataSama'])


In [ ]:
# Menghitung cosine similarity antara matriks TF-IDF
cosine_sim = cosine_similarity(tfidf_keterangan, tfidf_kata_sama)

# Dapatkan indeks kata yang paling mirip untuk setiap kalimat dari tf_user
indices = cosine_sim.argmax(axis=1)

# Buat kolom baru di tf_user untuk indeks kata yang paling mirip
tf_user['Most_Similar_Index'] = indices

# Tampilkan hasil
print(tf_user[['Keterangan', 'Most_Similar_Index']])

                   Keterangan  Most_Similar_Index
0       Pemasangan Pipa Sumur                   1
1          Servis Dipan Kasur                   4
2                  Cat Tembok                  10
3              Service Kulkas                   0
4    Design Pamflet dan Cetak                   8
..                        ...                 ...
155     Pemasangan Plat Mesin                  20
156         Pembuatakn Lemari                  25
157            Perawatan Pipa                   1
158            Perbaikan Pipa                   1
159           Cat Ulang Pintu                  24

[160 rows x 2 columns]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Misalnya, tfidf_vectorizer = TfidfVectorizer() dengan konfigurasi yang diperlukan
tfidf_vectorizer = TfidfVectorizer()

# Contoh penggunaan jika 'Kata Sama' dalam DataFrame df_average_time_diffs dijadikan sebagai fitur
kata_sama_texts = df_average_time_diffs['Kata Sama']

# Mengonversi teks menjadi vektor TF-IDF
tfidf_kata_sama = tfidf_vectorizer.fit_transform(kata_sama_texts)

In [ ]:
# Download stopwords jika belum
nltk.download('stopwords')

# Fungsi untuk preprocessing teks
def preprocess_text(text):
    words = nltk.word_tokenize(text.lower())
    words = [word for word in words if word.isalpha()]
    stop_words = set(stopwords.words('indonesian'))
    words = [word for word in words if word not in stop_words]
    processed_text = ' '.join(words)
    return processed_text

# Preprocessing teks dalam kolom 'Keterangan'
tf_user['Preprocessed_Keterangan'] = tf_user['Keterangan'].apply(preprocess_text)

# Kolom 'Kata Sama' di df_average_time_diffs akan digunakan sebagai referensi klaster
reference_clusters = df_average_time_diffs['Kata Sama']

# Inisialisasi objek TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Konversi teks yang sudah diproses menjadi representasi vektor TF-IDF
tfidf_keterangan = tfidf_vectorizer.fit_transform(tf_user['Preprocessed_Keterangan'])

# Inisialisasi list untuk menyimpan hasil klaster berdasarkan referensi 'Kata Sama'
cluster_labels = []

# Looping melalui setiap teks di tf_user untuk mencocokkan klaster berdasarkan 'Kata Sama'
for i in range(len(tf_user)):
    # Peroleh teks dari tf_user
    text = tf_user['Preprocessed_Keterangan'][i]

    # Konversi teks ke dalam vektor TF-IDF
    text_vector = tfidf_vectorizer.transform([text])

    # Hitung kemiripan antara teks dengan setiap klaster pada 'Kata Sama'
    similarities = [len(set(text.split()) & set(cluster.split())) for cluster in reference_clusters]

    # Tentukan klaster yang paling cocok (dengan kemiripan terbesar)
    best_cluster = similarities.index(max(similarities))

    # Tambahkan label klaster ke list cluster_labels
    cluster_labels.append(best_cluster)

# Tambahkan kolom 'Cluster' ke tf_user berdasarkan hasil pencocokan klaster
tf_user['Cluster'] = cluster_labels

# Tampilkan DataFrame tf_user dengan kolom Cluster
tf_user[['Keterangan', 'Cluster']]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Keterangan,Cluster
0,Pemasangan Pipa Sumur,1
1,Servis Dipan Kasur,4
2,Cat Tembok,10
3,Service Kulkas,0
4,Design Pamflet dan Cetak,6
...,...,...
155,Pemasangan Plat Mesin,20
156,Pembuatakn Lemari,25
157,Perawatan Pipa,1
158,Perbaikan Pipa,1


In [ ]:
# Membuat DataFrame contoh tf_user dan df_average_time_diffs
# Silakan gantikan ini dengan definisi sebenarnya dari tf_user dan df_average_time_diffs

# Mengkonversi kolom 'Timestamp' ke tipe data datetime
tf_user['Timestamp'] = pd.to_datetime(tf_user['Timestamp'])

# Fungsi untuk menambahkan waktu berdasarkan selisih waktu dari df_average_time_diffs
def add_time_by_cluster(row):
    cluster = row['Cluster']
    time_diff_seconds = df_average_time_diffs[df_average_time_diffs['Cluster'] == cluster]['Selisih Waktu'].values[0]
    time_diff = pd.Timedelta(seconds=time_diff_seconds)
    new_time = row['Timestamp'] + time_diff
    return new_time

# Menambahkan kolom baru 'Time Recomendation' dengan timestamp yang sudah ditambahkan waktu
tf_user['Time Recomendation'] = tf_user.apply(add_time_by_cluster, axis=1)

# Menampilkan tf_user setelah penambahan timestamp baru
tf_user


,Nama User,Daerah,Order,User Rating,Keterangan,Timestamp,Keyword,Row Number,Cleaned_Keterangan,Most_Similar_Index,Preprocessed_Keterangan,Cluster,Time Recomendation
0,Andi Susanto,Purwokerto,Ningsih Instalasi Pipa,2.8,Pemasangan Pipa Sumur,2023-12-20,Instalasi Pipa,1,pemasangan pipa sumur,1,pemasangan pipa sumur,1,2024-02-14 00:00:00
1,Andi Susanto,Purwokerto,Kurniawan Tukang Kayu,4.2,Servis Dipan Kasur,2023-08-07,Tukang Kayu,2,servis dipan kasur,4,servis dipan kasur,4,2024-01-12 00:00:00
2,Andi Susanto,Purwokerto,Lestari Tukang Bangunan dan Pengecetan,4.9,Cat Tembok,2023-05-03,Tukang Bangunan dan Pengecetan,3,cat tembok,10,cat tembok,10,2023-11-21 08:00:00
3,Andi Susanto,Purwokerto,Lestari Instalasi Listrik,4.1,Service Kulkas,2023-10-12,Instalasi Listrik,4,service kulkas,0,service kulkas,0,2024-03-13 00:00:00
4,Andi Susanto,Purwokerto,Sari Design dan Cetak,2.8,Design Pamflet dan Cetak,2023-11-08,Design dan Cetak,5,design pamflet cetak,8,design pamflet cetak,6,2024-02-26 18:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Siti Wati,Semarang,Utami Tukang Aluminium,4.7,Pemasangan Plat Mesin,2023-07-03,Tukang Aluminium,156,pemasangan plat mesin,20,pemasangan plat mesin,20,2023-11-21 00:00:00
156,Siti Wati,Semarang,Kusuma Tukang Kayu,2.8,Pembuatakn Lemari,2023-08-21,Tukang Kayu,157,pembuatakn lemari,25,pembuatakn lemari,25,2023-11-10 00:00:00
157,Siti Wati,Semarang,Susanto Perpipaan Gas,4.5,Perawatan Pipa,2023-09-20,Perpipaan Gas,158,perawatan pipa,1,perawatan pipa,1,2023-11-15 00:00:00
158,Siti Wati,Semarang,Hartono Instalasi Pipa,4.9,Perbaikan Pipa,2023-10-09,Instalasi Pipa,159,perbaikan pipa,1,perbaikan pipa,1,2023-12-04 00:00:00


In [ ]:
tf_user = tf_user.loc[:, ['Nama User','Daerah','Order', 'Timestamp', 'Time Recomendation']]
tf_user

,Nama User,Daerah,Order,Timestamp,Time Recomendation
0,Andi Susanto,Purwokerto,Ningsih Instalasi Pipa,2023-12-20,2024-02-14 00:00:00
1,Andi Susanto,Purwokerto,Kurniawan Tukang Kayu,2023-08-07,2024-01-12 00:00:00
2,Andi Susanto,Purwokerto,Lestari Tukang Bangunan dan Pengecetan,2023-05-03,2023-11-21 08:00:00
3,Andi Susanto,Purwokerto,Lestari Instalasi Listrik,2023-10-12,2024-03-13 00:00:00
4,Andi Susanto,Purwokerto,Sari Design dan Cetak,2023-11-08,2024-02-26 18:00:00
...,...,...,...,...,...
155,Siti Wati,Semarang,Utami Tukang Aluminium,2023-07-03,2023-11-21 00:00:00
156,Siti Wati,Semarang,Kusuma Tukang Kayu,2023-08-21,2023-11-10 00:00:00
157,Siti Wati,Semarang,Susanto Perpipaan Gas,2023-09-20,2023-11-15 00:00:00
158,Siti Wati,Semarang,Hartono Instalasi Pipa,2023-10-09,2023-12-04 00:00:00


In [ ]:
import pandas as pd
from datetime import datetime

# Misalkan tf_user adalah DataFrame yang berisi data dari CSV Anda
# Jika tidak terdefinisi sebelumnya, pastikan untuk membaca data dari CSV ke dalam tf_user terlebih dahulu

# Memasukkan nama dari user
nama_user = input("Masukkan nama user: ")

# Mencocokkan nama user
filtered_data = tf_user[tf_user['Nama User'] == nama_user]

# Mengurutkan hasil berdasarkan 'Time Recommendation' secara descending
filtered_data = filtered_data.sort_values(by='Time Recomendation', ascending=False)

# Mendapatkan tanggal hari ini
today = datetime.now().date()

# Memfilter 'Time Recommendation' setelah hari ini
filtered_data = filtered_data[pd.to_datetime(filtered_data['Time Recomendation']).dt.date > today]
filtered_data


Masukkan nama user: Siti Wati


,Nama User,Daerah,Order,Timestamp,Time Recomendation
153,Siti Wati,Semarang,Hadi Design dan Cetak,2023-12-05,2024-03-24 18:00:00
149,Siti Wati,Semarang,Gunawan Tukang Aluminium,2023-10-06,2024-03-11 06:00:00
150,Siti Wati,Semarang,Kurniawan Tukang AC,2023-10-01,2024-03-02 00:00:00
151,Siti Wati,Semarang,Sari Perpipaan Gas,2023-12-07,2024-02-01 00:00:00
159,Siti Wati,Semarang,Kusuma Tukang Kayu,2023-10-11,2023-12-31 00:00:00
